# EDA - Simplified Version

## Key Features:
1. `event_key` parsed directly from filename
2. Snow pixels are **included** in images, but tracked via `snow_frac`
3. Can filter by snow fraction threshold in analysis
4. Useful for winter storm events (e.g., 2021 Texas)

In [32]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, box
from rasterio.mask import mask

# 0) Path Configuration

In [33]:
BASE = Path("..")  # project/
DATA = BASE / "data"
RAW_IMGS = DATA / "raw" / "imgs"
PROCESSED = DATA / "processed"
RESULT = BASE / "result"
RESULT.mkdir(parents=True, exist_ok=True)

PLOT_DIR = RESULT / "plots"
PLOT_DIR.mkdir(parents=True, exist_ok=True)

# New v2 folders
IMG_DIRS = [
    RAW_IMGS / "GEE_VNP46A2_outages_v2_Dallas",
    RAW_IMGS / "GEE_VNP46A2_outages_v2_Harris",
]

CLOUD_SUMMARY = RAW_IMGS / "GEE_VNP46A2_outages_tables" / "cloud_fraction_summary_v2.csv"

# Parameters
PRE_DAYS = 5
POST_DAYS = 5
CLOUD_THRESHOLD = 0.3

# NEW: Snow filtering threshold
# Set to None to include all images regardless of snow
# Set to a value (e.g., 0.5) to exclude images with snow_frac > threshold
SNOW_THRESHOLD = None  # None = keep all, 0.5 = exclude if >50% snow

# 1) Read cloud summary

In [34]:
cloud = pd.read_csv(CLOUD_SUMMARY, parse_dates=["date"])
cloud["date"] = pd.to_datetime(cloud["date"]).dt.normalize()

# Calculate snow fraction
cloud["snow_frac"] = cloud["snow_px"] / cloud["base_valid_px"].clip(lower=1)

# Mark usable images (cloud filter)
cloud["usable"] = (
    (cloud["img_exists"] == True) &
    (cloud["cloud_frac"] >= 0) &
    (cloud["cloud_frac"] <= CLOUD_THRESHOLD) &
    (cloud["base_valid_px"] > 0)
)

# Optional: Apply snow filter
if SNOW_THRESHOLD is not None:
    cloud["usable"] = cloud["usable"] & (cloud["snow_frac"] <= SNOW_THRESHOLD)
    print(f"Snow threshold applied: <= {SNOW_THRESHOLD}")
else:
    print("Snow threshold: None (all snow levels included)")

print(f"\nCloud summary records: {len(cloud)}")
print(f"Usable records: {cloud['usable'].sum()}")

# Show snow statistics
print(f"\nSnow fraction statistics:")
print(cloud["snow_frac"].describe())

Snow threshold: None (all snow levels included)

Cloud summary records: 296
Usable records: 141

Snow fraction statistics:
count    296.000000
mean       0.036796
std        0.146075
min        0.000000
25%        0.000000
50%        0.000000
75%        0.001512
max        0.797108
Name: snow_frac, dtype: float64


# 2) Parse outage dates from event_key

In [35]:
def parse_event_key(event_key):
    """
    Parse event_key to extract county and outage dates
    Format: {county}_{startYYYYMMDDHHMM}_{endYYYYMMDDHHMM}
    """
    parts = event_key.split("_")
    county = parts[0]
    start_str = parts[1]
    end_str = parts[2]
    
    outage_start = pd.to_datetime(start_str, format="%Y%m%d%H%M").normalize()
    outage_end = pd.to_datetime(end_str, format="%Y%m%d%H%M").normalize()
    
    return county, outage_start, outage_end

# Apply parsing
parsed = cloud["event_key"].apply(parse_event_key)
cloud["county_parsed"] = parsed.apply(lambda x: x[0])
cloud["outage_start"] = parsed.apply(lambda x: x[1])
cloud["outage_end"] = parsed.apply(lambda x: x[2])

print("Parsed outage dates from event_key")
display(cloud[["event_key", "county", "outage_start", "outage_end"]].drop_duplicates().head())

Parsed outage dates from event_key


,event_key,county,outage_start,outage_end
0,Dallas_201505290500_201505300645,Dallas,2015-05-29,2015-05-30
12,Dallas_201504190045_201504200030,Dallas,2015-04-19,2015-04-20
24,Dallas_201505271900_201505281030,Dallas,2015-05-27,2015-05-28
36,Dallas_201504250015_201504251530,Dallas,2015-04-25,2015-04-25
47,Dallas_201505270015_201505271345,Dallas,2015-05-27,2015-05-27


# 3) Scan local images

In [36]:
# Filename pattern: {event_key}_{YYYYMMDD}_VNP46A2_ntl.tif
pat = re.compile(
    r"^([A-Za-z]+_\d{12}_\d{12})_(\d{8})_VNP46A2_ntl\.tif$", 
    re.I
)

img_records = []
for d in IMG_DIRS:
    if not d.exists():
        print(f"Warning: {d} does not exist")
        continue
    for fp in d.glob("*.tif"):
        m = pat.match(fp.name)
        if not m:
            continue
        event_key = m.group(1)
        date = pd.to_datetime(m.group(2), format="%Y%m%d").normalize()
        img_records.append({
            "event_key": event_key,
            "date": date,
            "path": fp
        })

imgs = pd.DataFrame(img_records)
print(f"Found {len(imgs)} images")

if imgs.empty:
    raise FileNotFoundError("No matching GeoTIFFs found")

Found 290 images


# 4) Merge with cloud summary

In [37]:
cloud_usable = cloud.loc[
    cloud["usable"], 
    ["event_key", "date", "county", "event_id", "outage_start", "outage_end", 
     "snow_px", "base_valid_px", "snow_frac"]
].copy()

imgs = imgs.merge(cloud_usable, on=["event_key", "date"], how="inner")
print(f"Images after merge: {len(imgs)}")

Images after merge: 141


# 5) Phase labeling

In [38]:
def label_phase(row):
    pre_start = row["outage_start"] - pd.Timedelta(days=PRE_DAYS)
    pre_end = row["outage_start"] - pd.Timedelta(days=1)
    post_start = row["outage_end"] + pd.Timedelta(days=1)
    post_end = row["outage_end"] + pd.Timedelta(days=POST_DAYS)
    
    if pre_start <= row["date"] <= pre_end:
        return "pre"
    elif row["outage_start"] <= row["date"] <= row["outage_end"]:
        return "during"
    elif post_start <= row["date"] <= post_end:
        return "post"
    return None

imgs["phase"] = imgs.apply(label_phase, axis=1)
imgs = imgs[imgs["phase"].isin(["pre", "during", "post"])].copy()
print(f"Images after phase labeling: {len(imgs)}")

Images after phase labeling: 141


# 6) Calculate mean NTL

In [39]:
def mean_ntl(tif_path: Path) -> float:
    with rasterio.open(tif_path) as src:
        arr = src.read(1).astype("float32")
        nodata = src.nodata
        if nodata is not None:
            arr[arr == nodata] = np.nan
        return float(np.nanmean(arr))

imgs["mean_ntl"] = imgs["path"].apply(mean_ntl)
print(f"Mean NTL calculated for {len(imgs)} images")

Mean NTL calculated for 141 images


# 7) Event availability summary

In [40]:
event_availability = (
    imgs.groupby("event_key")
        .agg(
            county=("county", "first"),
            event_id=("event_id", "first"),
            outage_start=("outage_start", "first"),
            outage_end=("outage_end", "first"),
            n_pre=("phase", lambda x: (x == "pre").sum()),
            n_during=("phase", lambda x: (x == "during").sum()),
            n_post=("phase", lambda x: (x == "post").sum()),
            total_images=("date", "count"),
            avg_snow_frac=("snow_frac", "mean"),
            max_snow_frac=("snow_frac", "max"),
        )
        .reset_index()
)

event_availability.to_csv(RESULT / "event_availability.csv", index=False)
print("Event availability:")
display(event_availability)

Event availability:


,event_key,county,event_id,outage_start,outage_end,n_pre,n_during,n_post,total_images,avg_snow_frac,max_snow_frac
0,Dallas_201504190045_201504200030,Dallas,Texas-2,2015-04-19,2015-04-20,1,1,2,4,0.000000e+00,0.000000
1,Dallas_201504250015_201504251530,Dallas,Texas-4,2015-04-25,2015-04-25,3,0,3,6,2.969397e-04,0.001782
2,Dallas_201505270015_201505271345,Dallas,Texas-5,2015-05-27,2015-05-27,1,0,2,3,5.641854e-04,0.001158
3,Dallas_201505271900_201505281030,Dallas,Texas-5,2015-05-27,2015-05-28,1,0,3,4,4.231391e-04,0.001158
4,Dallas_201505290500_201505300645,Dallas,Texas-6,2015-05-29,2015-05-30,1,0,5,6,2.820927e-04,0.001158
5,Dallas_201910210215_201910260645,Dallas,Texas-20,2019-10-21,2019-10-26,4,3,2,9,1.979598e-05,0.000178
6,Dallas_202008162100_202008181615,Dallas,Texas-14,2020-08-16,2020-08-18,4,1,4,9,0.000000e+00,0.000000
7,Dallas_202102140915_202102181515,Dallas,Texas-1,2021-02-14,2021-02-18,0,1,5,6,3.505579e-01,0.795344
8,Dallas_202102181530_202102211230,Dallas,Texas-1,2021-02-18,2021-02-21,1,3,4,8,2.629184e-01,0.795344
9,Dallas_202209042000_202209070545,Dallas,Texas-8,2022-09-04,2022-09-07,1,4,3,8,2.436216e-04,0.001158


# 8) Event x Phase statistics

In [41]:
event_phase = (
    imgs.groupby(["event_key", "phase"])
        .agg(
            county=("county", "first"),
            event_id=("event_id", "first"),
            min_ntl=("mean_ntl", "min"),
            mean_ntl=("mean_ntl", "mean"),
            n_images=("mean_ntl", "count"),
            phase_start=("date", "min"),
            phase_end=("date", "max"),
            outage_start=("outage_start", "first"),
            outage_end=("outage_end", "first"),
            avg_snow_frac=("snow_frac", "mean"),
        )
        .reset_index()
        .sort_values(["event_key", "phase"])
)

event_phase.to_csv(RESULT / "event_phase_stats.csv", index=False)
print("Event x Phase stats:")
display(event_phase)

Event x Phase stats:


,event_key,phase,county,event_id,min_ntl,mean_ntl,n_images,phase_start,phase_end,outage_start,outage_end,avg_snow_frac
0,Dallas_201504190045_201504200030,during,Dallas,Texas-2,4.435658,4.435658,1,2015-04-20,2015-04-20,2015-04-19,2015-04-20,0.000000e+00
1,Dallas_201504190045_201504200030,post,Dallas,Texas-2,3.283190,3.484235,2,2015-04-21,2015-04-22,2015-04-19,2015-04-20,0.000000e+00
2,Dallas_201504190045_201504200030,pre,Dallas,Texas-2,3.139886,3.139886,1,2015-04-17,2015-04-17,2015-04-19,2015-04-20,0.000000e+00
3,Dallas_201504250015_201504251530,post,Dallas,Texas-4,3.190229,3.581152,3,2015-04-26,2015-04-30,2015-04-25,2015-04-25,5.938794e-04
4,Dallas_201504250015_201504251530,pre,Dallas,Texas-4,3.283190,3.801376,3,2015-04-20,2015-04-22,2015-04-25,2015-04-25,0.000000e+00
5,Dallas_201505270015_201505271345,post,Dallas,Texas-5,3.233413,3.442474,2,2015-05-31,2015-06-01,2015-05-27,2015-05-27,2.672457e-04
6,Dallas_201505270015_201505271345,pre,Dallas,Texas-5,2.784106,2.784106,1,2015-05-25,2015-05-25,2015-05-27,2015-05-27,1.158065e-03
7,Dallas_201505271900_201505281030,post,Dallas,Texas-5,3.233413,3.521965,3,2015-05-31,2015-06-02,2015-05-27,2015-05-28,1.781638e-04
8,Dallas_201505271900_201505281030,pre,Dallas,Texas-5,2.784106,2.784106,1,2015-05-25,2015-05-25,2015-05-27,2015-05-28,1.158065e-03
9,Dallas_201505290500_201505300645,post,Dallas,Texas-6,3.233413,3.427079,5,2015-05-31,2015-06-04,2015-05-29,2015-05-30,1.068983e-04


# 9) Relative NTL change analysis

In [42]:
print("="*60)
print("Relative NTL Change Analysis (during vs pre)")
print("="*60)

results = []
for evt in event_phase["event_key"].unique():
    evt_df = event_phase[event_phase["event_key"] == evt]
    
    pre = evt_df[evt_df["phase"] == "pre"]
    during = evt_df[evt_df["phase"] == "during"]
    
    if len(pre) > 0 and len(during) > 0:
        pre_ntl = pre["mean_ntl"].values[0]
        during_ntl = during["mean_ntl"].values[0]
        pre_snow = pre["avg_snow_frac"].values[0]
        during_snow = during["avg_snow_frac"].values[0]
        
        change_pct = ((during_ntl - pre_ntl) / pre_ntl) * 100
        
        results.append({
            "event_key": evt,
            "county": pre["county"].values[0],
            "pre_ntl": round(pre_ntl, 2),
            "during_ntl": round(during_ntl, 2),
            "change_pct": round(change_pct, 1),
            "pre_snow_frac": round(pre_snow, 3),
            "during_snow_frac": round(during_snow, 3),
            "n_pre": pre["n_images"].values[0],
            "n_during": during["n_images"].values[0],
        })

change_df = pd.DataFrame(results)
print("\nNTL Change:")
display(change_df)

print(f"\nSummary:")
print(f"  Events with NTL decrease: {(change_df['change_pct'] < 0).sum()} / {len(change_df)}")
print(f"  Average change: {change_df['change_pct'].mean():.1f}%")
print(f"  Events with high snow (>10%): {((change_df['pre_snow_frac'] > 0.1) | (change_df['during_snow_frac'] > 0.1)).sum()}")

change_df.to_csv(RESULT / "ntl_change_analysis.csv", index=False)

Relative NTL Change Analysis (during vs pre)

NTL Change:


,event_key,county,pre_ntl,during_ntl,change_pct,pre_snow_frac,during_snow_frac,n_pre,n_during
0,Dallas_201504190045_201504200030,Dallas,3.14,4.44,41.3,0.000,0.000,1,1
1,Dallas_201910210215_201910260645,Dallas,4.04,3.86,-4.4,0.000,0.000,4,3
2,Dallas_202008162100_202008181615,Dallas,4.09,4.29,4.8,0.000,0.000,4,1
3,Dallas_202102181530_202102211230,Dallas,7.80,6.22,-20.2,0.795,0.435,1,3
4,Dallas_202209042000_202209070545,Dallas,4.71,4.04,-14.1,0.001,0.000,1,4
5,Dallas_202303030030_202303050145,Dallas,4.98,4.77,-4.1,0.000,0.000,1,2
6,Harris_201708281215_201709011700,Harris,3.28,4.59,40.1,0.005,0.000,1,1
7,Harris_201801152200_201801161500,Harris,3.90,3.45,-11.6,0.001,0.000,3,1
8,Harris_201801162215_201801171415,Harris,3.79,4.69,24.0,0.001,0.000,4,1
9,Harris_201801182015_201801190830,Harris,3.84,3.99,3.9,0.001,0.015,4,1



Summary:
  Events with NTL decrease: 8 / 17
  Average change: 5.8%
  Events with high snow (>10%): 1


# 10) Snow impact analysis

In [43]:
print("="*60)
print("Snow Impact Analysis")
print("="*60)

# Correlation between snow and NTL
corr = imgs[["snow_frac", "mean_ntl"]].corr().iloc[0, 1]
print(f"\nCorrelation (snow_frac vs mean_ntl): {corr:.3f}")

# Snow by phase
print("\nSnow fraction by phase:")
snow_by_phase = imgs.groupby("phase")["snow_frac"].agg(["mean", "std", "max", "count"])
print(snow_by_phase)

# High snow events
high_snow_events = change_df[
    (change_df["pre_snow_frac"] > 0.1) | (change_df["during_snow_frac"] > 0.1)
]
if len(high_snow_events) > 0:
    print(f"\nHigh snow events (snow_frac > 10%):")
    display(high_snow_events[["event_key", "change_pct", "pre_snow_frac", "during_snow_frac"]])
else:
    print("\nNo high snow events found.")

Snow Impact Analysis

Correlation (snow_frac vs mean_ntl): 0.764

Snow fraction by phase:
            mean       std       max  count
phase                                      
during  0.062542  0.204988  0.795344     34
post    0.023299  0.122532  0.786991     58
pre     0.017347  0.113483  0.795344     49

High snow events (snow_frac > 10%):


,event_key,change_pct,pre_snow_frac,during_snow_frac
3,Dallas_202102181530_202102211230,-20.2,0.795,0.435


# 11) Visualization: Time series

In [44]:
ts = (
    imgs.groupby(["event_key", "date"])
        .agg(
            mean_ntl=("mean_ntl", "mean"),
            snow_frac=("snow_frac", "mean"),
            county=("county", "first"),
            event_id=("event_id", "first"),
            outage_start=("outage_start", "first"),
            outage_end=("outage_end", "first"),
        )
        .reset_index()
        .sort_values(["event_key", "date"])
)

for event_key, g in ts.groupby("event_key"):
    g = g.sort_values("date")
    if g.empty:
        continue
    
    start = g["outage_start"].iloc[0]
    end = g["outage_end"].iloc[0]
    county = g["county"].iloc[0]
    event_id = g["event_id"].iloc[0]
    max_snow = g["snow_frac"].max()

    fig, ax1 = plt.subplots(figsize=(10, 5))
    
    # NTL line
    color1 = "tab:blue"
    ax1.plot(g["date"], g["mean_ntl"], marker="o", linewidth=2, color=color1, label="Mean NTL")
    ax1.set_xlabel("Date")
    ax1.set_ylabel("Mean NTL (nW/cm2/sr)", color=color1)
    ax1.tick_params(axis="y", labelcolor=color1)
    
    # Snow fraction on secondary axis (if any snow)
    if max_snow > 0.01:
        ax2 = ax1.twinx()
        color2 = "tab:cyan"
        ax2.fill_between(g["date"], 0, g["snow_frac"], alpha=0.3, color=color2, label="Snow frac")
        ax2.set_ylabel("Snow Fraction", color=color2)
        ax2.tick_params(axis="y", labelcolor=color2)
        ax2.set_ylim(0, max(0.5, max_snow * 1.2))
    
    # Outage period
    if pd.notna(start) and pd.notna(end):
        ax1.axvspan(start, end, alpha=0.2, color="red", label="Outage period")

    ax1.set_title(f"{county} | {event_id}\n{event_key}")
    ax1.grid(True, alpha=0.3)
    ax1.legend(loc="upper left")
    plt.xticks(rotation=45)
    plt.tight_layout()

    fname = f"{event_key}_timeseries.png"
    plt.savefig(PLOT_DIR / fname, dpi=200, bbox_inches="tight")
    plt.close()

print(f"Saved time series plots to: {PLOT_DIR}")

Saved time series plots to: ../result/plots


# 12) POI Buffer Analysis Setup

In [45]:
# POI data path
POI_CSV = DATA / "raw" / "POI" / "texas_critical_infra_points_2022.csv"

poi = pd.read_csv(POI_CSV)
poi = poi.rename(columns={"lon": "longitude", "lat": "latitude"})

gdf_poi = gpd.GeoDataFrame(
    poi,
    geometry=gpd.points_from_xy(poi["longitude"], poi["latitude"]),
    crs="EPSG:4326"
)

# Project to metric CRS (Texas Centric Albers)
gdf_poi = gdf_poi.to_crs(epsg=3083)

# 500m buffer
BUFFER_M = 500
gdf_poi["geometry"] = gdf_poi.geometry.buffer(BUFFER_M)

# Map city_source to county name
CITY_TO_COUNTY = {
    "Houston": "Harris",
    "Dallas": "Dallas",
}
gdf_poi["county"] = gdf_poi["city_source"].map(CITY_TO_COUNTY)
gdf_poi = gdf_poi[gdf_poi["county"].notna()].copy()

print(f"POI count: {len(gdf_poi)}")
print(f"Counties: {gdf_poi['county'].unique()}")

POI count: 620
Counties: ['Harris' 'Dallas']


# 13) Calculate Buffer vs Non-buffer NTL

In [46]:
def zonal_nanmean(src, geom):
    """Calculate nanmean within geometry"""
    try:
        out, _ = mask(src, [geom], crop=False, all_touched=True)
        arr = out[0].astype("float32")
        if src.nodata is not None:
            arr[arr == src.nodata] = np.nan
        return float(np.nanmean(arr))
    except Exception:
        return np.nan


# Create buffer union for each county
buffers_by_county = {
    c: gdf_poi[gdf_poi["county"] == c].geometry.unary_union
    for c in gdf_poi["county"].unique()
}

print(f"Buffer counties: {list(buffers_by_county.keys())}")

records = []
error_count = 0

for idx, r in imgs.iterrows():
    county = r["county"]
    tif = r["path"]

    if county not in buffers_by_county:
        continue

    try:
        with rasterio.open(tif) as src:
            extent = box(*src.bounds)

            # Project buffer to TIF's CRS
            buf_geom = gpd.GeoSeries(
                [buffers_by_county[county]], 
                crs=gdf_poi.crs
            ).to_crs(src.crs).iloc[0]

            # Non-facility area = extent - buffer
            nonbuf_geom = extent.difference(buf_geom)

            buf_mean = zonal_nanmean(src, buf_geom)
            nonbuf_mean = zonal_nanmean(src, nonbuf_geom) if not nonbuf_geom.is_empty else np.nan

        rec = dict(r)
        rec["mean_ntl_buffer"] = buf_mean
        rec["mean_ntl_nonbuffer"] = nonbuf_mean
        records.append(rec)

    except Exception as e:
        print(f"Error: {tif.name}: {e}")
        error_count += 1

print(f"\nProcessed: {len(records)}, Errors: {error_count}")

df_buf = pd.DataFrame(records)

Buffer counties: ['Harris', 'Dallas']


/var/folders/68/ttx4m32n6gl0cmfb5mv3fhwm0000gn/T/ipykernel_42578/2916729593.py:15: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  c: gdf_poi[gdf_poi["county"] == c].geometry.unary_union
/var/folders/68/ttx4m32n6gl0cmfb5mv3fhwm0000gn/T/ipykernel_42578/2916729593.py:15: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  c: gdf_poi[gdf_poi["county"] == c].geometry.unary_union



Processed: 141, Errors: 0


# 14) Buffer vs Non-buffer Statistics

In [47]:
event_phase_buffer = (
    df_buf.groupby(["event_key", "phase"])
          .agg(
              county=("county", "first"),
              buffer_min=("mean_ntl_buffer", "min"),
              buffer_mean=("mean_ntl_buffer", "mean"),
              nonbuffer_min=("mean_ntl_nonbuffer", "min"),
              nonbuffer_mean=("mean_ntl_nonbuffer", "mean"),
              n_images=("mean_ntl_buffer", "count"),
              avg_snow_frac=("snow_frac", "mean"),
              outage_start=("outage_start", "first"),
              outage_end=("outage_end", "first"),
          )
          .reset_index()
)

event_phase_buffer.to_csv(RESULT / "event_phase_buffer_vs_nonbuffer.csv", index=False)
print("Event x Phase x Buffer stats:")
display(event_phase_buffer)

Event x Phase x Buffer stats:


,event_key,phase,county,buffer_min,buffer_mean,nonbuffer_min,nonbuffer_mean,n_images,avg_snow_frac,outage_start,outage_end
0,Dallas_201504190045_201504200030,during,Dallas,0.796216,0.796216,4.364083,4.364083,1,0.000000e+00,2015-04-19,2015-04-20
1,Dallas_201504190045_201504200030,post,Dallas,0.584471,0.602080,3.219729,3.420433,2,0.000000e+00,2015-04-19,2015-04-20
2,Dallas_201504190045_201504200030,pre,Dallas,0.588621,0.588621,3.070220,3.070220,1,0.000000e+00,2015-04-19,2015-04-20
3,Dallas_201504250015_201504251530,post,Dallas,0.596392,0.663510,3.134558,3.518442,3,5.938794e-04,2015-04-25,2015-04-25
4,Dallas_201504250015_201504251530,pre,Dallas,0.584471,0.666792,3.219729,3.734983,3,0.000000e+00,2015-04-25,2015-04-25
5,Dallas_201505270015_201505271345,post,Dallas,0.590853,0.638226,3.164905,3.368381,2,2.672457e-04,2015-05-27,2015-05-27
6,Dallas_201505270015_201505271345,pre,Dallas,0.441398,0.441398,2.730058,2.730058,1,1.158065e-03,2015-05-27,2015-05-27
7,Dallas_201505271900_201505281030,post,Dallas,0.590853,0.653588,3.164905,3.447023,3,1.781638e-04,2015-05-27,2015-05-28
8,Dallas_201505271900_201505281030,pre,Dallas,0.441398,0.441398,2.730058,2.730058,1,1.158065e-03,2015-05-27,2015-05-28
9,Dallas_201505290500_201505300645,post,Dallas,0.590853,0.643778,3.164905,3.352951,5,1.068983e-04,2015-05-29,2015-05-30


# 15) Buffer vs Non-buffer Change Analysis

In [48]:
print("="*60)
print("Buffer vs Non-buffer Relative Change (during vs pre)")
print("="*60)

buf_results = []
for evt in event_phase_buffer["event_key"].unique():
    evt_df = event_phase_buffer[event_phase_buffer["event_key"] == evt]
    
    pre = evt_df[evt_df["phase"] == "pre"]
    during = evt_df[evt_df["phase"] == "during"]
    
    if len(pre) > 0 and len(during) > 0:
        pre_buf = pre["buffer_mean"].values[0]
        during_buf = during["buffer_mean"].values[0]
        pre_nonbuf = pre["nonbuffer_mean"].values[0]
        during_nonbuf = during["nonbuffer_mean"].values[0]
        
        buf_change = ((during_buf - pre_buf) / pre_buf) * 100 if pre_buf > 0 else np.nan
        nonbuf_change = ((during_nonbuf - pre_nonbuf) / pre_nonbuf) * 100 if pre_nonbuf > 0 else np.nan
        
        buf_results.append({
            "event_key": evt,
            "county": pre["county"].values[0],
            "buf_change_pct": round(buf_change, 1) if not np.isnan(buf_change) else np.nan,
            "nonbuf_change_pct": round(nonbuf_change, 1) if not np.isnan(nonbuf_change) else np.nan,
            "diff": round(buf_change - nonbuf_change, 1) if not (np.isnan(buf_change) or np.isnan(nonbuf_change)) else np.nan,
            "pre_snow": round(pre["avg_snow_frac"].values[0], 3),
            "during_snow": round(during["avg_snow_frac"].values[0], 3),
        })

buf_change_df = pd.DataFrame(buf_results)
print("\nBuffer vs Non-buffer change:")
display(buf_change_df)

print(f"\nInterpretation:")
print(f"  diff > 0: Buffer dropped LESS than non-buffer (facilities more resilient)")
print(f"  diff < 0: Buffer dropped MORE than non-buffer")
print(f"\nEvents where buffer more resilient: {(buf_change_df['diff'] > 0).sum()} / {len(buf_change_df)}")

buf_change_df.to_csv(RESULT / "buffer_change_analysis.csv", index=False)

Buffer vs Non-buffer Relative Change (during vs pre)

Buffer vs Non-buffer change:


,event_key,county,buf_change_pct,nonbuf_change_pct,diff,pre_snow,during_snow
0,Dallas_201504190045_201504200030,Dallas,35.3,42.1,-6.9,0.000,0.000
1,Dallas_201910210215_201910260645,Dallas,4.5,-4.3,8.8,0.000,0.000
2,Dallas_202008162100_202008181615,Dallas,5.7,4.9,0.8,0.000,0.000
3,Dallas_202102181530_202102211230,Dallas,-21.4,-20.2,-1.2,0.795,0.435
4,Dallas_202209042000_202209070545,Dallas,-7.1,-14.4,7.3,0.001,0.000
5,Dallas_202303030030_202303050145,Dallas,13.3,-3.7,17.0,0.000,0.000
6,Harris_201708281215_201709011700,Harris,49.6,40.5,9.1,0.005,0.000
7,Harris_201801152200_201801161500,Harris,-10.7,-11.7,1.0,0.001,0.000
8,Harris_201801162215_201801171415,Harris,21.8,24.1,-2.2,0.001,0.000
9,Harris_201801182015_201801190830,Harris,2.3,3.9,-1.6,0.001,0.015



Interpretation:
  diff > 0: Buffer dropped LESS than non-buffer (facilities more resilient)
  diff < 0: Buffer dropped MORE than non-buffer

Events where buffer more resilient: 9 / 17


# 16) Buffer vs Non-buffer Time Series Plots

In [49]:
PLOT_DIR_BUF = RESULT / "plots_buffer"
PLOT_DIR_BUF.mkdir(parents=True, exist_ok=True)

ts_buf = (
    df_buf.groupby(["event_key", "date"])
          .agg(
              buffer=("mean_ntl_buffer", "mean"),
              nonbuffer=("mean_ntl_nonbuffer", "mean"),
              snow_frac=("snow_frac", "mean"),
              county=("county", "first"),
              event_id=("event_id", "first"),
              outage_start=("outage_start", "first"),
              outage_end=("outage_end", "first"),
          )
          .reset_index()
          .sort_values(["event_key", "date"])
)

plot_count = 0
for event_key, g in ts_buf.groupby("event_key"):
    g = g.sort_values("date")
    if g.empty or g["buffer"].isna().all():
        continue

    start = g["outage_start"].iloc[0]
    end = g["outage_end"].iloc[0]
    county = g["county"].iloc[0]
    event_id = g["event_id"].iloc[0]

    fig, ax1 = plt.subplots(figsize=(10, 5))
    
    ax1.plot(g["date"], g["buffer"], marker="o", linewidth=2, 
             color="tab:orange", label="Critical facility buffer (1km)")
    ax1.plot(g["date"], g["nonbuffer"], marker="s", linewidth=2, 
             color="tab:blue", label="Non-facility area")
    
    if pd.notna(start) and pd.notna(end):
        ax1.axvspan(start, end, alpha=0.2, color="red", label="Outage period")

    ax1.set_xlabel("Date")
    ax1.set_ylabel("Mean NTL (nW/cm2/sr)")
    ax1.set_title(f"{county} | {event_id}\n{event_key}\nBuffer vs Non-buffer")
    ax1.grid(True, alpha=0.3)
    ax1.legend(loc="upper left")
    plt.xticks(rotation=45)
    plt.tight_layout()

    fname = f"{event_key}_buffer_vs_nonbuffer.png"
    plt.savefig(PLOT_DIR_BUF / fname, dpi=200, bbox_inches="tight")
    plt.close()
    plot_count += 1

print(f"Saved {plot_count} buffer plots to: {PLOT_DIR_BUF}")

Saved 23 buffer plots to: ../result/plots_buffer
